# I. Setup

This notebook runs the conversion from ASL and T1w images to PET images using a Pix2Pix architecture similar to LowGAN

## Cloning the Pix2Pix Repository and installing libraries

In [ ]:
%%bash
git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
cd pytorch-CycleGAN-and-pix2pix
pip install -r requirements.txt
pip install antspyx
pip install antspynet
pip install torchio
pip install unet
pip install opencv-python

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
Updating files: 100% (74/74), done.


  Using cached visdom-0.2.4-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 34.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 29.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 17.8 MB/s eta 0:00:00


Load required packages

In [2]:
import ants
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import antspynet

2024-01-18 18:57:41.253001: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 18:57:43.433944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# II. Create pix2pix compatible data

The MRI has to be reshaped to be pix2pix compatible.

In [14]:
data_source = '/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/'
subjects = os.listdir(data_source)
subjects = [x for x in subjects if (x[0]!='.') and ('derivative' not in x)]

# pick the test set subjects such that they are balanced for left and right TLE
subjects_testing = ['sub-RID0681', 'sub-RID0791', 'sub-RID0663', 'sub-RID0576',
       'sub-RID0656', 'sub-RID0408', 'sub-RID0784', 'sub-RID0341',
       'sub-RID0420', 'sub-RID0595', 'sub-RID0449', 'sub-RID0665',
       'sub-RID0433', 'sub-RID0670', 'sub-RID0679', 'sub-RID0680',
       'sub-RID0522', 'sub-RID0598', 'sub-RID0455', 'sub-RID0572']

subjects_training = set(subjects)-set(subjects_testing)

subjects = list(subjects_training) + list(subjects_testing)

### Functions to load, reshape and rescale the images

First we rescale and reshape the images. The original data had skullstripped inputs, so these functions assume a skullstripped input, however, it is not necessary for the code to run successfully.

In [4]:
## Run this set of functions if you want to make the output image a square

def resample_image(img):
    # use 1mm isotropic resampled image
    image = img.copy()

    # Get the original voxel spacing.
    original_spacing = image.spacing

    # Calculate the new voxel spacing.
    new_spacing = [1, 1, 1]
  
    # Resample the image to 1mm isotropic.
    resampled_image = ants.resample_image(image, new_spacing, interp_type=0)

    return resampled_image

def calculate_mask_centroid(image):
    # Load the image data
    image_mask = (image>0)

    # Calculate the centroid coordinates
    indices = np.indices(image_mask.shape)
    centroid = np.mean(indices[:, image_mask.astype(bool)], axis=1)

    return centroid

def zero_pad(img, pad=50):
    x,y,z = img.shape
    img_padded = np.ones((x+pad,y+pad, z+pad))*img[0,0,0]
    img_padded[pad//2:pad//2 + x, pad//2:pad//2 + y, pad//2:pad//2 + z] = img
    return img_padded

def reshape_to_square2(img, edge=220):
    img_pad = zero_pad(img)
    x,y,z = img_pad.shape
    x_start = (x-edge)//4
    y_start = (y-edge)//2
    z_start = (z-edge)//4

    y_shift = 0
    x_shift = 30
    z_shift = 0

    img_reshaped = img_pad[x_start+x_shift:x_start+edge+x_shift, y_start+y_shift:y_start+edge+y_shift, z_start+z_shift:z_shift+z_start+edge ]
    return img_reshaped

def make_cube(img):
    # this function zero pads the image until it becomes a cube
    x,y,z = img.shape
    max_dim = np.max(img.shape)
    
    to_add_x = (max_dim-x)//2
    to_add_y = (max_dim-y)//2
    to_add_z = (max_dim-z)//2
    
    zero_padded = np.ones((x+(to_add_x*2), y+(to_add_y*2), z+(to_add_z*2)))*img[0,0,0]
    
    zero_padded[to_add_x:x+to_add_x, to_add_y:y+to_add_y, to_add_z:z+to_add_z] = img
    
    return zero_padded

def norm_zero_to_one(vol3D):
    '''Normalize a 3D volume to zero to one range

    Parameters:
      vol3D (3D numpy array): 3D image volume
    '''

    # normalize to 0 to 1 range
    vol3D = vol3D - np.min(vol3D) # set lower bound
    vol3D = vol3D / ( np.max(vol3D) - np.min(vol3D) ) # set upper bound

    return make_cube(vol3D)


def load_and_skullstrip_lofi_list(subject):
    t1 = ants.image_read(os.path.join(data_source,subject,subject+'_ses-research3T_space-T1w_desc-gaussian-3.0_cbf.nii.gz'))
    t2 = ants.image_read(os.path.join(data_source,subject,subject+'_ses-research3T_space-T1w_desc-gaussian-1.0_cbf.nii.gz'))
    flair = ants.image_read(os.path.join(data_source, subject,subject+'_ses-research3T_T1w.nii.gz'))

    brain_mask = ants.image_read(os.path.join(data_source, subject,subject+'_ses-research3T_T1w_brain_mask.nii.gz'))
    
    subject_dict = []
    
    print(t1.shape)
    print(t2.shape)
    print(flair.shape)
    print(brain_mask.shape)
    
    subject_dict.append(norm_zero_to_one((t1).numpy()))
    subject_dict.append(norm_zero_to_one((t2).numpy()))
    subject_dict.append(norm_zero_to_one((flair).numpy()))
    
    return np.array(subject_dict)


def load_hifi_list(subject):
    t1 = ants.image_read(os.path.join(data_source,subject,subject+'_ses-clinical01_pet.nii.gz'))
    t2 = ants.image_read(os.path.join(data_source,subject,subject+'_ses-clinical01_pet.nii.gz'))
    flair = ants.image_read(os.path.join(data_source, subject,subject+'_ses-clinical01_pet.nii.gz'))

    brain_mask = ants.image_read(os.path.join(data_source, subject,subject+'_ses-research3T_T1w_brain_mask.nii.gz'))
    
    subject_dict = []
    
    print(t1.shape)
    print(t2.shape)
    print(flair.shape)
    print(brain_mask.shape)
    
    subject_dict.append(norm_zero_to_one((t1).numpy()))
    subject_dict.append(norm_zero_to_one((t2).numpy()))
    subject_dict.append(norm_zero_to_one((flair).numpy()))
    
    return np.array(subject_dict)

Performing the skullstripping and reshaping

In [573]:
subjects

['sub-RID0415',
 'sub-RID0450',
 'sub-RID0585',
 'sub-RID0662',
 'sub-RID0591',
 'sub-RID0396',
 'sub-RID0672',
 'sub-RID0596',
 'sub-RID0659',
 'sub-RID0658',
 'sub-RID0860',
 'sub-RID0574',
 'sub-RID0693',
 'sub-RID0448',
 'sub-RID0666',
 'sub-RID0440',
 'sub-RID0646',
 'sub-RID0429',
 'sub-RID0660',
 'sub-RID0445',
 'sub-RID0675',
 'sub-RID0592',
 'sub-RID0664',
 'sub-RID0454',
 'sub-RID0671',
 'sub-RID0597',
 'sub-RID0620',
 'sub-RID0904',
 'sub-RID0571',
 'sub-RID0365',
 'sub-RID0619',
 'sub-RID0798',
 'sub-RID0490',
 'sub-RID0697',
 'sub-RID0925',
 'sub-RID0785',
 'sub-RID0673',
 'sub-RID0583',
 'sub-RID0578',
 'sub-RID0652',
 'sub-RID0383',
 'sub-RID0688',
 'sub-RID0372',
 'sub-RID0380',
 'sub-RID0648',
 'sub-RID0512',
 'sub-RID0587',
 'sub-RID0689',
 'sub-RID0668',
 'sub-RID0681',
 'sub-RID0791',
 'sub-RID0663',
 'sub-RID0576',
 'sub-RID0656',
 'sub-RID0408',
 'sub-RID0784',
 'sub-RID0341',
 'sub-RID0420',
 'sub-RID0595',
 'sub-RID0449',
 'sub-RID0665',
 'sub-RID0433',
 'sub-RI

In [16]:
train_dir = os.path.join(data_source,'..','ml_data','resliced_reshaped_v2')


In [ ]:
# create reshape and skullstrip the registered data
orientation='axial'
train_dir = os.path.join(data_source,'..','ml_data','resliced_reshaped_v2')

if os.path.exists(train_dir)==False:
    os.makedirs(train_dir)

for sub in subjects:
    print(sub)
    # lofi
    
    a = load_and_skullstrip_lofi_list(sub)
    
    # Visualize the Skullstripped Images
    print(sub)
    print(a.shape)
    plt.figure()
    plt.subplot(231)
    plt.imshow(a[0,:,:,150], cmap='gray')
    plt.subplot(232)
    plt.title('Skullstripped LOFI - '+sub)
    plt.imshow(a[1,:,:,150], cmap='gray')
    plt.subplot(233)
    plt.imshow(a[2,:,:,150], cmap='gray')

    plt.subplot(234)
    plt.imshow(a[0,:,150,:], cmap='gray')
    plt.subplot(235)
    plt.imshow(a[1,:,150,:], cmap='gray')
    plt.subplot(236)
    plt.imshow(a[2,:,150,:], cmap='gray')

    np.save(os.path.join(train_dir,sub+'_V0_affine.npy'), a.astype('float32'))

    #hifi
    a = load_hifi_list(sub)
    


    # Visualize the Skullstripped Images
    plt.figure()
    plt.subplot(231)
    plt.imshow(a[0,:,:,60], cmap='gray')
    plt.subplot(232)
    plt.title('Skullstripped HIFI - '+sub)
    plt.imshow(a[1,:,:,60], cmap='gray')
    plt.subplot(233)
    plt.imshow(a[2,:,:,60], cmap='gray')
    plt.subplot(234)
    plt.imshow(a[0,:,60,:], cmap='gray')
    plt.subplot(235)
    plt.imshow(a[1,:,60,:], cmap='gray')
    plt.subplot(236)
    plt.imshow(a[2,:,60,:], cmap='gray')

    np.save(os.path.join(train_dir,sub+'_V1_affine.npy'), a.astype('float32'))

sub-RID0659
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0659
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0380
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0380
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0420
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0420
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0672
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0672
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0791
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0791
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0595
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0595
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 

/tmp/ipykernel_25677/1069658996.py:17: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.



(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0675
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0675
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0450
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0450
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0512
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0512
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0415
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0415
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0688
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0688
(3, 320, 320, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-RID0785
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
(208, 300, 320)
sub-R

### Actually doing the loading and creating a pix2pix dataset

In [17]:
source_data_folder = train_dir


In [ ]:

## This cell creartes the Train/Test/Validation datasets given a source data folder specified above

# Specify normalization - If brain_norm=True, then it will do intensity normalization to the average whole brain intensity
brain_norm = True

# iterate through all three planes and create the three datasets
for index_for_slicing, plane in zip([1,2,0],['coronal','axial','sagittal']):
    dataset_path = os.path.join(data_source,'derivatives','ML_datasets','reshaped_square_v3_'+plane) # Name of the generated dataset of png files.
    subs_not_processed = []

    # Check if there dataset path folder exists, if not, create it

    if os.path.exists(dataset_path)==False:
        os.makedirs(dataset_path)

    # train/val/test split
    N_train = int(len(subjects)*0.7)
    N_val = int(len(subjects)*0)
    N_test = int(len(subjects)*0.3)

    print('Number of training examples: ', N_train)
    print('Number of validation examples: ', N_val)
    print('Number of testing examples: ', N_test)

    N = 0 # counter for each subject iteration

    # Define the number of slices and the spacing between slices
    n_slices = 3
    spacing = 5

    # Functions to deal with edge cases

    # Function for concatenating the T1, T2 and FLAIR
    def concat_t1_t2_flair(t1,t2,flair):
        return np.concatenate([t1[:,:,np.newaxis],t2[:,:,np.newaxis],flair[:,:,np.newaxis]],axis=2)

    # process all subjects
    for subID in subjects:

        print(subID)

        try:

            # assign subject to proper set
            N += 1 # increase counter
            if N <= N_train:
                ttv_set = 'train'
            elif N <= N_train + N_val:
                ttv_set = 'val'
            else:
                ttv_set = 'test'

            # preprocess and save out paired lofi/hifi datasets of PNG files
            for scan in ['hifi','lofi']:
                for flip in [False]:
                    # select appropriate volume
                    if scan == 'hifi':
                        array = np.load(os.path.join(source_data_folder,subID+'_V1_affine.npy'))*255
                        output_path = os.path.join(dataset_path, 'A', ttv_set)

                    else:
                        array = np.load(os.path.join(source_data_folder,subID+'_V0_affine.npy'))*255
                        output_path = os.path.join(dataset_path, 'B', ttv_set)

                    os.makedirs( output_path, exist_ok=True) # make output directory is they don't already exist

                    z = array.shape[2] # process along the z-axis


                    for i in range(0,z):
                        if flip:
                            filename = subID + '_' + str(i) + '_flip.png'
                        else:
                            filename = subID + '_' + str(i) + '.png'

                        filepath = os.path.join(output_path, filename)
                        if index_for_slicing == 0:
                            png3 = concat_t1_t2_flair(array[0,i,:,:], array[1,i,:,:], array[2,i,:,:])
                        elif index_for_slicing == 1:
                            png3 = concat_t1_t2_flair(array[0,:,i,:], array[1,:,i,:], array[2,:,i,:])
                        elif index_for_slicing == 2:
                            png3 = concat_t1_t2_flair(array[0,:,:,i], array[1,:,:,i], array[2,:,:,i])

                        cv2.imwrite(filepath, png3)

        except Exception as e:
            print(e)
            subs_not_processed.append(subID)

Number of training examples:  48
Number of validation examples:  0
Number of testing examples:  20
sub-RID0415
sub-RID0450
sub-RID0585
sub-RID0662
sub-RID0591
sub-RID0396
sub-RID0672
sub-RID0596
sub-RID0659
sub-RID0658
sub-RID0860
sub-RID0574
sub-RID0693
sub-RID0448
sub-RID0666
sub-RID0440
sub-RID0646
sub-RID0429
sub-RID0660
sub-RID0445
sub-RID0675
sub-RID0592
sub-RID0664
sub-RID0454
sub-RID0671
sub-RID0597
sub-RID0620
sub-RID0904
sub-RID0571
sub-RID0365
sub-RID0619
sub-RID0798
sub-RID0490
sub-RID0697
sub-RID0925
sub-RID0785
sub-RID0673
sub-RID0583
sub-RID0578
sub-RID0652
sub-RID0383
sub-RID0688
sub-RID0372
sub-RID0380
sub-RID0648
sub-RID0512
sub-RID0587
sub-RID0689
sub-RID0668
sub-RID0681
sub-RID0791
sub-RID0663
sub-RID0576
sub-RID0656
sub-RID0408
sub-RID0784
sub-RID0341


In [ ]:
train_subjects = subjects[N_train]
val_subjects = subjects[N_train:N_train+N_val]
test_subjects = subjects[-N_test:]

Do final conversion to Pix2pix format using the built-in pix2pix functions

In [ ]:
# format dataset for pix2pix input
for plane in ['coronal','axial','sagittal']:
    dataset_path = os.path.join(data_source,'derivatives','ML_datasets','reshaped_square_v3_'+plane) # Name of the generated dataset of png files.

    fold_A_name = os.path.join(dataset_path,'A')
    fold_B_name = os.path.join(dataset_path,'B')
    output_name = dataset_path+'_pix2pix'

    !python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
    --fold_A $fold_A_name \
    --fold_B $fold_B_name \
    --fold_AB $output_name

# III.a. Run the Stage 1 pix2pix model

In [ ]:
checkpoints_dir = '/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/outputs/pix2pix_checkpoints'
planes = ['axial','coronal','sagittal']
model_stem = 'asl_to_pet_reshaped_v3_bs2_100epochs_'

In [ ]:

for plane in planes:
    dataset_path = os.path.join(data_source,'derivatives','ML_datasets','reshaped_square_v3_'+plane)
    # set model name
    model_name = model_stem+plane
    dataset_name = dataset_path+'_pix2pix'
    # train model
    !python pytorch-CycleGAN-and-pix2pix/train.py \
    --dataroot $dataset_name \
    --name $model_name \
    --model pix2pix \
    --direction BtoA \
    --preprocess resize_and_crop \
    --save_epoch_freq 20 \
    --display_id -1 \
    --lr 0.0001 \
    --n_epochs 30 \
    --n_epochs_decay 70 \
    --netG unet_256 \
    --batch_size 2  

# III.b. Run the Stage 1 pix2pix model on the test set

In [29]:
planes = ['axial','coronal','sagittal']
phase = 'test'
for plane in planes:
    # set model name
    model_name = model_stem+plane
    
    # directory where outputs will be placed in
    results_dir = os.path.join(data_source, 'derivatives','ML_outputs',model_name)

    dataset_name = os.path.join(data_source, 'derivatives','ML_datasets','reshaped_square_v3_'+plane+'_pix2pix')
    # run model on test set
    !python pytorch-CycleGAN-and-pix2pix/test.py \
    --dataroot $dataset_name \
    --direction BtoA \
    --model pix2pix \
    --name $model_name \
    --phase $phase \
    --preprocess scale_width \
    --epoch latest \
    --num_test 100000 \
    --checkpoints_dir ./checkpoints \
    --results_dir $results_dir

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix	[default: None]
             dataset_mode: aligned                       
                direction: BtoA                          	[default: AtoB]
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
      

processing (0125)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0341_210.png']
processing (0130)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0341_215.png']
processing (0135)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0341_22.png']
processing (0140)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0341_224.png']
processing (0145)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0341_229.png']
processing 

processing (0335)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_111.png']
processing (0340)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_116.png']
processing (0345)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_120.png']
processing (0350)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_125.png']
processing (0355)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_13.png']
processing 

processing (0545)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_300.png']
processing (0550)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_305.png']
processing (0555)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_31.png']
processing (0560)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_314.png']
processing (0565)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0408_319.png']
processing 

processing (0755)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0420_201.png']
processing (0760)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0420_206.png']
processing (0765)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0420_210.png']
processing (0770)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0420_215.png']
processing (0775)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_axial_pix2pix/test/sub-RID0420_22.png']
processing 

...

processing (5520)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_170.png']
processing (5525)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_175.png']
processing (5530)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_18.png']
processing (5535)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_184.png']
processing (5540)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_189.pn

processing (5725)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_68.png']
processing (5730)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_72.png']
processing (5735)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_77.png']
processing (5740)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_81.png']
processing (5745)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0681_86.png']


processing (5930)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0784_251.png']
processing (5935)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0784_256.png']
processing (5940)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0784_260.png']
processing (5945)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0784_265.png']
processing (5950)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0784_27.pn

processing (6135)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_148.png']
processing (6140)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_152.png']
processing (6145)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_157.png']
processing (6150)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_161.png']
processing (6155)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_166.p

processing (6340)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_45.png']
processing (6345)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_5.png']
processing (6350)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_54.png']
processing (6355)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_59.png']
processing (6360)-th image... ['/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_datasets/reshaped_square_v3_sagittal_pix2pix/test/sub-RID0791_63.png']
p

## Convert Stage 1 test set output to NIFTI

Functions for NIFTI conversion

In [30]:
import cv2

In [31]:
def get_volume_predicted(subject, pix2pix_out_dir, plane, model_name='', phase='test', epoch='latest'):
    dicom = np.zeros((320,320,320))
    
    for i in range(320):
        try:
            file_stem = subject+'_'+str(i)
            recon_filename = os.path.join(pix2pix_out_dir, model_name, phase+'_' + str(epoch),'images',file_stem + '_fake_B.png')
            img = cv2.imread(recon_filename)[:,:,0]
            res = cv2.resize(img, dsize=(320, 320), interpolation=cv2.INTER_CUBIC)

            if plane=='axial':
                dicom[:,:,i] = res
            elif plane=='sagittal':
                dicom[i,:,:] = res
            elif plane=='coronal':
                dicom[:,i,:] = res
        except Exception as e:
            print(e)
            continue
    return np.array(dicom)


Running the nifti conversion

In [32]:
# get the affine from the original images
import nibabel as nib
affine = nib.load(os.path.join(data_source, subjects[-13],subjects[-13]+'_ses-research3T_T1w.nii.gz')).affine

def save_nifti(img, affine, fname):
    nib.save(nib.Nifti1Image(img, affine),fname)
    

In [33]:
from tqdm import tqdm
import ants
import matplotlib.pyplot as plt

phase = 'test'
planes = ['axial','coronal','sagittal']


for plane in planes:
    # set model name
    model_name = model_stem+plane
    
    pix2pix_out_dir = os.path.join(data_source, 'derivatives','ML_outputs', model_name)

    volume_output_dir = os.path.join(data_source, 'derivatives','ML_outputs','recon_volumes', model_name)

    try:
        os.makedirs(volume_output_dir)
    except:
        print('Directory exists...')

    # Save the reconstructed images
    for sub in tqdm(test_subjects):

        vol = get_volume_predicted(sub, pix2pix_out_dir=pix2pix_out_dir,  plane=plane ,phase=phase, model_name=model_name)

        # crop the image to the original dimensions, remove the zero padding
        pet = vol[56:-56,10:-10,:]

        save_nifti(pet,affine,os.path.join(volume_output_dir,sub+'_recon_pet.nii.gz'))



Directory exists...


 85%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                   | 17/20 [00:55<00:09,  3.03s/it][ WARN:0@334023.319] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_axial/asl_to_pet_reshaped_v3_bs2_100epochs_axial/test_latest/images/sub-RID0598_0_fake_B.png'): can't open/read file: check file path/integrity
[ WARN:0@334023.320] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_axial/asl_to_pet_reshaped_v3_bs2_100epochs_axial/test_latest/images/sub-RID0598_1_fake_B.png'): can't open/read file: check file path/integrity
[ WARN:0@334023.334] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_da

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not 

[ WARN:0@334023.508] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_axial/asl_to_pet_reshaped_v3_bs2_100epochs_axial/test_latest/images/sub-RID0598_248_fake_B.png'): can't open/read file: check file path/integrity
[ WARN:0@334023.509] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_axial/asl_to_pet_reshaped_v3_bs2_100epochs_axial/test_latest/images/sub-RID0598_249_fake_B.png'): can't open/read file: check file path/integrity
[ WARN:0@334023.510] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_axial/asl_to_pet_reshaped_v3_bs2_100epochs_axial/test_latest/images/sub-RID0598_250_

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not 

 85%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                   | 17/20 [00:41<00:06,  2.29s/it][ WARN:0@334072.134] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_coronal/asl_to_pet_reshaped_v3_bs2_100epochs_coronal/test_latest/images/sub-RID0598_0_fake_B.png'): can't open/read file: check file path/integrity
[ WARN:0@334072.135] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_coronal/asl_to_pet_reshaped_v3_bs2_100epochs_coronal/test_latest/images/sub-RID0598_1_fake_B.png'): can't open/read file: check file path/integrity
[ WARN:0@334072.135] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/s

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not 

 85%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                   | 17/20 [00:38<00:06,  2.21s/it][ WARN:0@334116.816] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_sagittal/asl_to_pet_reshaped_v3_bs2_100epochs_sagittal/test_latest/images/sub-RID0598_0_fake_B.png'): can't open/read file: check file path/integrity
[ WARN:0@334116.817] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_PET/source_data/BIDS_pet_t1_asl/derivatives/ML_outputs/asl_to_pet_reshaped_v3_bs2_100epochs_sagittal/asl_to_pet_reshaped_v3_bs2_100epochs_sagittal/test_latest/images/sub-RID0598_1_fake_B.png'): can't open/read file: check file path/integrity
[ WARN:0@334116.817] global loadsave.cpp:248 findDecoder imread_('/mnt/leif/littlab/users/allucas/CNT_borel/Q20_ASL_P

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:43<00:00,  2.18s/it]


### Remove the vertical lines that are present in the multi-plane models

I will try different approaches: multi-plane averaging and anisotropic diffusion smoothing

In [34]:
import numpy
def anisotropic_diffusion(img, niter=1, kappa=80, gamma=0.1, voxelspacing=(0.8,0.8,0.8), option=3):
    r"""
    Edge-preserving, XD Anisotropic diffusion.


    Parameters
    ----------
    img : array_like
        Input image (will be cast to numpy.float).
    niter : integer
        Number of iterations.
    kappa : integer
        Conduction coefficient, e.g. 20-100. ``kappa`` controls conduction
        as a function of the gradient. If ``kappa`` is low small intensity
        gradients are able to block conduction and hence diffusion across
        steep edges. A large value reduces the influence of intensity gradients
        on conduction.
    gamma : float
        Controls the speed of diffusion. Pick a value :math:`<= .25` for stability.
    voxelspacing : tuple of floats or array_like
        The distance between adjacent pixels in all img.ndim directions
    option : {1, 2, 3}
        Whether to use the Perona Malik diffusion equation No. 1 or No. 2,
        or Tukey's biweight function.
        Equation 1 favours high contrast edges over low contrast ones, while
        equation 2 favours wide regions over smaller ones. See [1]_ for details.
        Equation 3 preserves sharper boundaries than previous formulations and
        improves the automatic stopping of the diffusion. See [2]_ for details.

    Returns
    -------
    anisotropic_diffusion : ndarray
        Diffused image.

    Notes
    -----
    Original MATLAB code by Peter Kovesi,
    School of Computer Science & Software Engineering,
    The University of Western Australia,
    pk @ csse uwa edu au,
    <http://www.csse.uwa.edu.au>

    Translated to Python and optimised by Alistair Muldal,
    Department of Pharmacology,
    University of Oxford,
    <alistair.muldal@pharm.ox.ac.uk>

    Adapted to arbitrary dimensionality and added to the MedPy library Oskar Maier,
    Institute for Medical Informatics,
    Universitaet Luebeck,
    <oskar.maier@googlemail.com>

    June 2000  original version. -
    March 2002 corrected diffusion eqn No 2. -
    July 2012 translated to Python -
    August 2013 incorporated into MedPy, arbitrary dimensionality -

    References
    ----------
    .. [1] P. Perona and J. Malik.
       Scale-space and edge detection using ansotropic diffusion.
       IEEE Transactions on Pattern Analysis and Machine Intelligence,
       12(7):629-639, July 1990.
    .. [2] M.J. Black, G. Sapiro, D. Marimont, D. Heeger
       Robust anisotropic diffusion.
       IEEE Transactions on Image Processing,
       7(3):421-432, March 1998.
    """
    # define conduction gradients functions
    if option == 1:
        def condgradient(delta, spacing):
            return numpy.exp(-(delta/kappa)**2.)/float(spacing)
    elif option == 2:
        def condgradient(delta, spacing):
            return 1./(1.+(delta/kappa)**2.)/float(spacing)
    elif option == 3:
        kappa_s = kappa * (2**0.5)

        def condgradient(delta, spacing):
            top = 0.5*((1.-(delta/kappa_s)**2.)**2.)/float(spacing)
            return numpy.where(numpy.abs(delta) <= kappa_s, top, 0)

    # initialize output array
    out = numpy.array(img, dtype=numpy.float32, copy=True)

    # set default voxel spacing if not supplied
    if voxelspacing is None:
        voxelspacing = tuple([1.] * img.ndim)

    # initialize some internal variables
    deltas = [numpy.zeros_like(out) for _ in range(out.ndim)]

    for _ in range(niter):

        # calculate the diffs
        for i in range(out.ndim):
            slicer = tuple([slice(None, -1) if j == i else slice(None) for j in range(out.ndim)])
            deltas[i][slicer] = numpy.diff(out, axis=i)

        # update matrices
        matrices = [condgradient(delta, spacing) * delta for delta, spacing in zip(deltas, voxelspacing)]

        # subtract a copy that has been shifted ('Up/North/West' in 3D case) by one
        # pixel. Don't as questions. just do it. trust me.
        for i in range(out.ndim):
            slicer = tuple([slice(1, None) if j == i else slice(None) for j in range(out.ndim)])
            matrices[i][slicer] = numpy.diff(matrices[i], axis=i)

        # update the image
        out += gamma * (numpy.sum(matrices, axis=0))

    return out

In [35]:
for sub in tqdm(test_subjects):
    try:
        # load the axial image
        volume_output_dir = os.path.join(data_source, 'derivatives','ML_outputs','recon_volumes', model_stem+'axial')
        path_img = os.path.join(volume_output_dir,sub+'_recon_pet.nii.gz')
        axial = anisotropic_diffusion(nib.load(path_img).get_fdata(),40)

        # load the axial image
        volume_output_dir = os.path.join(data_source, 'derivatives','ML_outputs','recon_volumes', model_stem+'axial')
        path_img = os.path.join(volume_output_dir,sub+'_recon_pet.nii.gz')
        coronal = anisotropic_diffusion(nib.load(path_img).get_fdata(),40)

        # load the axial image
        volume_output_dir = os.path.join(data_source, 'derivatives','ML_outputs','recon_volumes', model_stem+'axial')
        path_img = os.path.join(volume_output_dir,sub+'_recon_pet.nii.gz')
        sagittal = anisotropic_diffusion(nib.load(path_img).get_fdata(),40)

        avg_planes = (axial+coronal+sagittal)/3
        out_dir = os.path.join(data_source, 'derivatives','ML_outputs','recon_volumes', model_stem+'merged_diffusion')
        if os.path.exists(out_dir)==False:
            os.makedirs(out_dir)
        save_nifti(avg_planes, affine, os.path.join(out_dir,sub+'_recon_pet.nii.gz'))
    except Exception as e:
        print(e)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [24:08<00:00, 72.42s/it]


### Registration to the original PET image

The outputs are slightly mis-aligned with the input images. To fix this, a rigid registration using mutual information and greedy will be used.

In [142]:
greedy_path = '/mnt/leif/littlab/tools/bin/greedy'
for plane in ['axial','coronal','sagittal']:
    for i in range(13):

        out_dir = os.path.join(data_source, 'derivatives','ML_outputs','recon_volumes', model_stem+plane)
        subject = subjects[-13+i]

        recon_pet = os.path.join(out_dir,subject+'_recon_pet.nii.gz')
        original_pet = os.path.join(data_source,subject,subject+'_ses-clinical01_pet.nii.gz')

        registration_matrix = os.path.join(out_dir, subject+'_recon_to_original_pet.mat')

        recon_pet_registered = os.path.join(out_dir, subject+'_recon_pet_registered.nii.gz')
        # run the Greedy registration
        !$greedy_path -d 3 -a -i $original_pet $recon_pet -ia-image-centers -dof 6 -n 100x100x0x0 -m NMI -o $registration_matrix

        # apply the Greedy registration
        !$greedy_path -d 3 -rf $original_pet -rm $recon_pet $recon_pet_registered -r $registration_matrix


Executing with the default number of threads: 64
*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -10000   GNORM= 1.2558e-16
*************************************************
   I   NFN    FUNC        GNORM       STEPLENGTH
   1    3    -10000.000       0.000  45771658781.149
 THE MINIMIZATION TERMINATED WITHOUT DETECTING ERRORS.
Final RAS Transform: 
1 0 0 44.8 
0 1 0 8.04211 
0 0 1 0 
0 0 0 1 

*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -10000   GNORM= 2.25091e-16
*************************************************
   I   NFN    FUNC        GNORM       STEPLENGTH
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 6 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
1 0 0 44.8 
0 1 0 8.04211 
0 0 1 0 
0 0 0 1 

Final RAS Trans

  16   24    -11902.660       0.150       0.349
  17   26    -11902.660       0.167       0.182
  18   28    -11902.660       0.147       0.307
  19   30    -11902.660       0.201       0.248
  20   31    -11902.660       0.154       1.000
  21   32    -11902.660       0.130       1.000
  22   33    -11902.660       0.163       1.000
  23   36    -11902.660       0.129       0.062
  24   39    -11902.660       0.240       0.055
  25   40    -11902.660       0.196       1.000
  26   41    -11902.660       0.136       1.000
  27   42    -11902.661       0.139       1.000
  28   44    -11902.661       0.167       0.349
  29   46    -11902.661       0.128       0.205
  30   47    -11902.661       0.119       1.000
  31   51    -11902.661       0.103       0.418
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final

   4    5    -11965.307       2.345       1.000
   5    6    -11965.340       1.183       1.000
   6    7    -11965.341       0.929       1.000
   7    8    -11965.350       0.431       1.000
   8   10    -11965.352       0.413       0.445
   9   11    -11965.353       0.289       1.000
  10   13    -11965.353       0.492       0.307
  11   15    -11965.353       0.343       0.257
  12   16    -11965.353       0.188       1.000
  13   20    -11965.353       0.240       0.294
  14   21    -11965.353       0.223       1.000
  15   25    -11965.353       0.303       0.032
  16   26    -11965.353       0.199       1.000
  17   27    -11965.353       0.251       1.000
  18   28    -11965.353       0.126       1.000
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
0.999982 -0.00459924 0.00372472

   3    4    -12184.853       0.744       1.000
   4    5    -12184.865       0.392       1.000
   5    6    -12184.869       0.347       1.000
   6    7    -12184.869       0.172       1.000
   7    8    -12184.870       0.081       1.000
   8   10    -12184.870       0.045       0.438
   9   11    -12184.870       0.047       1.000
  10   13    -12184.870       0.034       0.482
  11   14    -12184.870       0.029       1.000
  12   15    -12184.870       0.035       1.000
  13   17    -12184.870       0.013       0.373
  14   18    -12184.870       0.016       1.000
  15   20    -12184.870       0.015       0.116
  16   21    -12184.870       0.019       1.000
  17   23    -12184.870       0.027       0.175
  18   25    -12184.870       0.026       0.139
  19   26    -12184.870       0.031       1.000
  20   27    -12184.870       0.016       1.000
  21   28    -12184.870       0.018       1.000
  22   29    -12184.870       0.015       1.000
  23   30    -12184.870       0.013     

Executing with the default number of threads: 64
Executing with the default number of threads: 64
*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -11765.9   GNORM= 31.2277
*************************************************
   I   NFN    FUNC        GNORM       STEPLENGTH
   1    2    -11788.406      16.135       0.032
   2    3    -11799.658       2.136       1.000
   3    4    -11799.717       2.008       1.000
   4    5    -11799.775       0.427       1.000
   5    6    -11799.779       0.375       1.000
   6    8    -11799.780       0.326       0.123
   7   10    -11799.780       0.353       0.230
   8   11    -11799.781       0.364       1.000
   9   13    -11799.781       0.325       0.194
  10   14    -11799.781       0.260       1.000
  11   15    -11799.781       0.184       1.000
  12   17    -11799.781       0.262       0.522
  13   18    -11799.781       0.309       1.000
  14   19    -11799.781       0.147       1.000

...

 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
0.999999 0.000144344 -0.00163108 3.67668 
-0.000161655 0.999944 -0.0106181 -11.3133 
0.00162946 0.0106183 0.999942 8.0524 
0 0 0 1 

Final RAS Transform: 
0.999999 0.000144344 -0.00163108 3.67668 
-0.000161655 0.999944 -0.0106181 -11.3133 
0.00162946 0.0106183 0.999942 8.0524 
0 0 0 1 

Final RAS Transform: 
0.999999 0.000144344 -0.00163108 3.67668 
-0.000161655 0.999944 -0.0106181 -11.3133 
0.00162946 0.0106183 0.999942 8.0524 
0 0 0 1 

Executing with the default number of threads: 64
Executing with the default number of threads: 64
*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -11928.6   GNORM= 21.2699
*************************************************
   I   NFN    FUNC        GN

  14   16    -12045.883       0.019       1.000
  15   17    -12045.883       0.009       1.000
  16   18    -12045.883       0.008       1.000
  17   19    -12045.883       0.009       1.000
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 6 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
0.999996 -0.00245614 0.00169259 -2.57489 
0.00245556 0.999997 0.000348737 -5.19368 
-0.00169344 -0.000344579 0.999999 4.40382 
0 0 0 1 

Final RAS Transform: 
0.999996 -0.00245614 0.00169259 -2.57489 
0.00245556 0.999997 0.000348737 -5.19368 
-0.00169344 -0.000344579 0.999999 4.40382 
0 0 0 1 

Final RAS Transform: 
0.999996 -0.00245614 0.00169259 -2.57489 
0.00245556 0.999997 0.000348737 -5.19368 
-0.00169344 -0.000344579 0.999999 4.40382 
0 0 0 1 

Executing with the default number of threads: 64
Executing with the default number of threads: 64
******

  20   29    -12408.466       0.025       0.151
  21   30    -12408.466       0.015       1.000
  22   33    -12408.466       0.015       0.041
  23   34    -12408.466       0.017       1.000
  24   36    -12408.466       0.009       0.289
  25   37    -12408.466       0.014       1.000
  26   38    -12408.466       0.014       1.000
  27   41    -12408.466       0.016       0.025
  28   44    -12408.466       0.009       0.089
  29   46    -12408.466       0.014       5.000
  30   47    -12408.466       0.021       1.000
  31   53    -12408.466       0.008       0.234
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
1 0.000920851 0.000254259 2.32611 
-0.000922542 0.999977 0.00673279 -7.71671 
-0.000248053 -0.00673303 0.999977 -11.0639 
0 0 0 1 

Final RAS Transform: 
1 0.000920851 0.00025

 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
0.999999 -0.000969568 -0.00055549 -2.02138 
0.000917829 0.996244 -0.0865868 10.6687 
0.000637356 0.0865863 0.996244 -17.1845 
0 0 0 1 

Final RAS Transform: 
0.999999 -0.000969568 -0.00055549 -2.02138 
0.000917829 0.996244 -0.0865868 10.6687 
0.000637356 0.0865863 0.996244 -17.1845 
0 0 0 1 

Final RAS Transform: 
0.999999 -0.000969568 -0.00055549 -2.02138 
0.000917829 0.996244 -0.0865868 10.6687 
0.000637356 0.0865863 0.996244 -17.1845 
0 0 0 1 

Executing with the default number of threads: 64
Executing with the default number of threads: 64
*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -11711.9   GNORM= 22.1377
*************************************************
   I   NFN    FUNC 

  24   40    -12150.574       0.017       0.189
  25   42    -12150.574       0.007       0.187
  26   44    -12150.574       0.018       0.396
  27   47    -12150.574       0.023       0.015
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
0.999999 -7.74269e-05 -0.00128275 -0.908272 
7.93269e-05 0.999999 0.00148117 -0.0987849 
0.00128264 -0.00148128 0.999998 -14.1001 
0 0 0 1 

Final RAS Transform: 
0.999999 -7.74269e-05 -0.00128275 -0.908272 
7.93269e-05 0.999999 0.00148117 -0.0987849 
0.00128264 -0.00148128 0.999998 -14.1001 
0 0 0 1 

Final RAS Transform: 
0.999999 -7.74269e-05 -0.00128275 -0.908272 
7.93269e-05 0.999999 0.00148117 -0.0987849 
0.00128264 -0.00148128 0.999998 -14.1001 
0 0 0 1 

Executing with the default number of threads: 64
Executing with the default number of thread

Registration of the smoothed PET images

In [36]:
greedy_path = '/mnt/leif/littlab/tools/bin/greedy'
for subject in test_subjects:

    out_dir = os.path.join(data_source, 'derivatives','ML_outputs','recon_volumes', model_stem+'merged_diffusion')

    recon_pet = os.path.join(out_dir,subject+'_recon_pet.nii.gz')
    original_pet = os.path.join(data_source,subject,subject+'_ses-clinical01_pet.nii.gz')
    
    registration_matrix = os.path.join(out_dir, subject+'_recon_to_original_pet.mat')
    
    recon_pet_registered = os.path.join(out_dir, subject+'_recon_pet_registered.nii.gz')
    # run the Greedy registration
    !$greedy_path -d 3 -a -i $original_pet $recon_pet -ia-image-centers -dof 6 -n 100x100x0x0 -m NMI -o $registration_matrix

    # apply the Greedy registration
    !$greedy_path -d 3 -rf $original_pet -rm $recon_pet $recon_pet_registered -r $registration_matrix
    
    

Executing with the default number of threads: 64
*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -11682.8   GNORM= 13.4723
*************************************************
   I   NFN    FUNC        GNORM       STEPLENGTH
   1    2    -11683.595      10.899       0.074
   2    3    -11686.254       1.627       1.000
   3    4    -11686.289       0.867       1.000
   4    5    -11686.299       0.501       1.000
   5    6    -11686.302       0.207       1.000
   6    7    -11686.303       0.393       1.000
   7    8    -11686.304       0.229       1.000
   8    9    -11686.304       0.315       1.000
   9   11    -11686.304       0.299       0.273
  10   13    -11686.305       0.273       0.218
  11   16    -11686.305       0.347       0.044
  12   19    -11686.305       0.212       0.097
  13   21    -11686.305       0.226       5.000
  14   22    -11686.305       0.220       1.000
  15   24    -11686.305       0.181       0.197


  18   23    -11953.749       0.022       0.131
  19   24    -11953.749       0.021       1.000
  20   26    -11953.749       0.018       0.336
  21   27    -11953.749       0.013       1.000
  22   28    -11953.749       0.009       1.000
  23   30    -11953.749       0.013       0.289
  24   31    -11953.749       0.009       1.000
  25   32    -11953.749       0.008       1.000
  26   34    -11953.749       0.010       0.111
  27   35    -11953.749       0.006       1.000
  28   36    -11953.749       0.006       1.000
  29   37    -11953.749       0.009       1.000
  30   38    -11953.749       0.007       1.000
  31   40    -11953.749       0.007       0.189
  32   42    -11953.749       0.006       0.397
  33   44    -11953.749       0.007       0.281
  34   47    -11953.749       0.007       0.029
  35   48    -11953.749       0.007       1.000
  36   49    -11953.749       0.006       1.000
  37   51    -11953.749       0.007       0.103
  38   52    -11953.749       0.006     

Executing with the default number of threads: 64
Executing with the default number of threads: 64
*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -12265.3   GNORM= 15.8488
*************************************************
   I   NFN    FUNC        GNORM       STEPLENGTH
   1    3    -12268.918       4.676       0.027
   2    4    -12269.214       1.474       1.000
   3    5    -12269.257       1.528       1.000
   4    7    -12269.289       0.777       0.520
   5    8    -12269.302       0.707       1.000
   6   10    -12269.307       0.567       0.071
   7   11    -12269.308       0.405       1.000
   8   13    -12269.308       0.414       0.163
   9   15    -12269.309       0.330       0.375
  10   16    -12269.309       0.566       1.000
  11   18    -12269.309       0.349       0.367
  12   20    -12269.309       0.131       0.343
  13   22    -12269.309       0.266       0.294
  14   25    -12269.310       0.251       0.194

  30   48    -12013.492       0.007       0.189
  31   49    -12013.492       0.005       1.000
  32   52    -12013.492       0.010       0.034
  33   53    -12013.492       0.010       1.000
  34   55    -12013.492       0.005       0.247
  35   57    -12013.492       0.005       0.179
  36   58    -12013.492       0.006       1.000
  37   59    -12013.492       0.010       1.000
  38   61    -12013.492       0.005       0.229
  39   62    -12013.492       0.010       1.000
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
0.999999 0.00030015 -0.00156058 3.58519 
-0.000299091 1 0.000678755 -4.58092 
0.00156079 -0.000678287 0.999999 9.85296 
0 0 0 1 

Final RAS Transform: 
0.999999 0.00030015 -0.00156058 3.58519 
-0.000299091 1 0.000678755 -4.58092 
0.00156079 -0.000678287 0.999999 9.85296 

  14   19    -11898.815       0.081       1.000
  15   20    -11898.815       0.114       1.000
  16   21    -11898.815       0.069       1.000
  17   23    -11898.815       0.092       0.260
  18   25    -11898.815       0.098       0.380
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
0.999993 0.00321691 -0.0016798 0.0130562 
-0.00322836 0.999971 -0.00685651 0.260403 
0.0016577 0.00686189 0.999975 -0.884182 
0 0 0 1 

*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -12014   GNORM= 29.4796
*************************************************
   I   NFN    FUNC        GNORM       STEPLENGTH
   1    2    -12027.995       7.614       0.034
   2    3    -12028.920       3.259       1.000
   3    4    -12029.245       1.154       1.000
  

   6    7    -12139.301       0.323       1.000
   7    8    -12139.303       0.073       1.000
   8    9    -12139.303       0.085       1.000
   9   10    -12139.303       0.046       1.000
  10   11    -12139.303       0.026       1.000
  11   13    -12139.303       0.026       0.417
  12   15    -12139.303       0.017       0.262
  13   16    -12139.303       0.019       1.000
  14   18    -12139.303       0.016       0.447
  15   19    -12139.303       0.015       1.000
  16   20    -12139.303       0.008       1.000
  17   22    -12139.303       0.007       0.052
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final RAS Transform: 
0.999999 0.000782195 -0.0014608 2.36342 
-0.000763042 0.999914 0.0130664 1.21289 
0.0014709 -0.0130653 0.999914 12.5384 
0 0 0 1 

Final RAS Transform: 
0.999999 0.000782195 -

Executing with the default number of threads: 64
Executing with the default number of threads: 64
*************************************************
  N=6   NUMBER OF CORRECTIONS=5       INITIAL VALUES F= -12007   GNORM= 28.6152
*************************************************
   I   NFN    FUNC        GNORM       STEPLENGTH
   1    2    -12025.636      11.832       0.035
   2    3    -12030.284       4.195       1.000
   3    4    -12030.505       3.568       1.000
   4    5    -12030.629       0.663       1.000
   5    6    -12030.640       0.471       1.000
   6    7    -12030.641       0.395       1.000
   7    8    -12030.642       0.259       1.000
   8   10    -12030.642       0.200       0.209
   9   11    -12030.643       0.105       1.000
  10   12    -12030.643       0.226       1.000
  11   14    -12030.643       0.204       0.514
  12   16    -12030.643       0.092       0.398
  13   18    -12030.643       0.168       0.313
  14   20    -12030.643       0.051       0.330
 

  14   21    -11891.830       0.177       0.036
  15   24    -11891.830       0.183       0.128
  16   28    -11891.830       0.233       0.041
  17   30    -11891.830       0.135       0.253
  18   33    -11891.830       0.137       0.776
  19   35    -11891.830       0.106       5.000
  20   39    -11891.830       0.102       1.372
  21   42    -11891.830       0.203       0.022
  22   45    -11891.830       0.111       1.947
  23   46    -11891.830       0.153       1.000
  24   48    -11891.830       0.152       0.223
  25   49    -11891.830       0.153       1.000
  26   51    -11891.830       0.160       5.000
  27   54    -11891.830       0.205       0.035
  28   55    -11891.830       0.116       1.000
  29   57    -11891.830       0.235       0.331
 IFLAG= -1  LINE SEARCH FAILED. SEE DOCUMENTATION OF ROUTINE MCSRCH ERROR RETURN OF LINE SEARCH: INFO= 3 POSSIBLE CAUSES: FUNCTION OR GRADIENT ARE INCORRECT OR INCORRECT TOLERANCESvnl_lbfgs: Error. Netlib routine lbfgs failed.
Final

  16   25    -11470.588       0.275       0.399
  17   28    -11470.588       0.342       0.157
  18   30    -11470.588       0.390       0.292
  19   31    -11470.588       0.336       1.000
  20   33    -11470.588       0.224       0.224
  21   34    -11470.589       0.151       1.000
  22   36    -11470.589       0.094       0.329
  23   37    -11470.589       0.150       1.000
  24   38    -11470.589       0.173       1.000
  25   39    -11470.589       0.161       1.000
  26   40    -11470.589       0.135       1.000
  27   41    -11470.589       0.205       1.000
  28   43    -11470.589       0.214       0.363
  29   44    -11470.589       0.278       1.000
  30   46    -11470.589       0.214       0.504
  31   47    -11470.589       0.243       1.000
  32   48    -11470.589       0.138       1.000
  33   50    -11470.589       0.148       0.268
  34   52    -11470.589       0.158       0.212
  35   53    -11470.589       0.148       1.000
  36   55    -11470.589       0.128     